# 빠른 결과를 위한 페이지를 적게 조정 하였음. 작업시 페이지 조정할것.

# NFT 토큰 (ERC-721)  거래 데이터 수집 및 저장 함수
- https://etherscan.io/tokentxns-nft

In [1]:
def make_token721txns():  
    import time
    import datetime
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    import re 
    
    Txn_Hash = []
    DateTime = []
    From = [] 
    To = [] 
    TokenID = []
    Token=[]
    cnt = 0 
    namelist = ['Txn_Hash','DateTime','From','To','TokenID','Token']
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36 Edg/95.0.1020.53'}
    
#     for idx in range(1,201): # 1~ 200 p
    # 페이지만 수정 
    for idx in range(1,2):
        url = 'https://etherscan.io/tokentxns-nft?p='+str(idx)
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')    
        for i in range(0,50):          
            Txn_Hash.append( soup.select('tbody>tr')[i].select('td')[1].get_text() )
            time.sleep(1)
            # Date Time (UTC)        
            DateTime.append(soup.select('tbody>tr')[i].select('td')[2].get_text() )
            time.sleep(1)
            # From
#     soup.select('tbody >tr')[i].select('td')[4].get_text() 
#     soup.select('tbody >tr')[i].select('td')[4].select('a')[0]['href'][9:]
            a = soup.select('tbody >tr')[i].select('td')[4].select('a')[0]['href'][9:]
            a = re.sub('#tokentxnsErc721', '' ,  str(a) ) 
            From.append(a)
            time.sleep(1)
    
            # To 
#     soup.select('tbody >tr')[i].select('td')[6].get_text()
            b = soup.select('tbody >tr')[i].select('td')[6].select('a')[0]['href'][9:]
            b = re.sub('#tokentxnsErc721', '' ,  str(b) )
            To.append(b)
            time.sleep(1)
            #tokenId
            TokenID.append(soup.select('tbody >tr')[i].select('td')[7].get_text()   )
            time.sleep(1)
            #token
# print(soup.select('tbody >tr')[1].select('td')[8].select('span')[0]['title'])
    
            try:   x = soup.select('tbody >tr')[i].select('td')[8].select('span[title]')[0]['title']
            except:  x =  soup.select('tbody >tr')[i].select('td')[8].get_text()
            Token.append(x)
            time.sleep(2)    
            
    token721txns = pd.DataFrame(data= [Txn_Hash, DateTime, From, To, TokenID, Token]).T
    token721txns.columns = namelist
    
     # csv 이름 설정 
    now = datetime.datetime.now()
    now = now.strftime('%Y_%m_%d_%H_%M_%S')
    csvname = 'data/token721txns/token721txns_' + now + '.csv'

    token721txns.to_csv(csvname)
    
    return print('함수 종료')
# null값이 있는 경우 => 이 주소는 어떤 사용자도 소유하지 않으며 종종 토큰 소각 및 발행/제네시스 이벤트와 연결되며 일반 null 주소로 사용됩니다.    

In [ ]:
make_token721txns()

# 오류
- 토큰 네임이 없는경우 
- 툴팁에 주소가 있는경우
- 토큰 네임이 길어서 짤린경우 

각각 모두 예외 처리 두었음.